# Lecture 6: Risk Measures

The previous lecture covered risk mapping, which should output a loss distribution $L_{t+1}. $ This lecture will be conerned with how to make such a distribution useful by introducing several methods for measuring risk. Such methods are used in financial institutions to perform tasks such as pricing insurance premiums, managing assets and determining how much capital one can afford to deploy on an investment given its risk. 

The main topics of discussion are:

 1. Basics of risk measurement
 2. Axiomization of risk measure theory
 3. Important risk measures
 4. Standard techniques for calculating VaR
 


In [15]:
#Importing standard libraries

import numpy as np
from matplotlib import pyplot as plt 
import scipy as sci
import pandas as pd

## Basics of Risk Measurement
There are several ways to measure risk. In fact, we have already covered a few examples throughout this course. For example, the option _greeks_ are measures of an option price's sensitivities to various factors, which is essentially a set of risk measures. However, the greeks are limited in the sense where they cannot be aggrigated (or be combined into a single measure), which is usually necessary in risk management. Another notable risk measure are the scenario-based risk measures.

For example, consider a set of risk factor changes $\chi = \{\vec{x}_1, \dots, \vec{x}_n\}$ and a set of weights $\vec{w} = \{w_1, \dots, w_n\} \in [0,1]^n$. Given a portfolio with an associated loss operator $l_{[t]}$, the risk is now taken to be the following function: 

$$\Psi_{[\chi,w]} = \max\{w_1l_{[t]}(\vec{x_1}),\dots, w_nl_{[t]}(\vec{x_n})\}$$

In other words, the risk is taken to be the maximum loss adjusted for its weight (probability of occurence). Whilst this approach doesn't tend to be highly looked up upon, it is very practical and tends to be used by exchanges such as the Chicago Mercantile Exchange and the Hong Kong Exchanges and Clearing which both use it to determine the margins for traders (see SPAN). 

The problems with scenario-based measures is that they are not derived from any theory and that it is usually difficult to accurately determine the weights. For this reason, we will stick to the more common approach of defining risk measures based on loss distributions. One can justify this method with the following statements:

 - Losses are the focus of risk management. Certain moments of distributions can provide monetary indicators
 - Loss distributions can be summarized on each _aggrigation level_ (individual investment, portfolio, company)
 - Diversification effects can be modeled
 - Loss distributions can be compared with eachother
 
And the two disadvantages being 1. you need a lot of data and 2. you may have incorrect assumptions, fit the data to the wrong model, use wrong risk mapping, etc. Although one can easily tell that the disadvantages are not in the the method itself but how it is implemented (i.e one must be very careful with implementing loss-distribution). Another important point is that it is generally bad practice to set the loss distribution as a risk measure (think about presenting a distribution to a CEO -- he doesn't care, he wants a number, eg. risk is $100MM). For this reason, one must then think about _what_ they want from a loss distribution. For example, an insurance business will likely be concerned with the expected value and/or median of this distribution.



#### Python Implementation

Given that there isn't much python implementation one can do for this section, a good idea would be to create a function that maps an input loss operator `l_t(x)`, weight vector `w` and risk factor change `chi` to an output risk measure `Phi` that assumes a scenario based model.

In [1]:
def Phi(l_t, w, chi):
    risks = [w[i]*l_t(chi[i]) for i in range(len(w))]
    return max(risks)

## Axiomatic Risk Measure Theory

The following section is all based on Artzner et al. (1999) which is linked in the "Further Reading" section. The goal is to define valid or _coherent_ risk measures using a set of axioms, and to use those set of axioms to judge commonly used risk measures such as VaR and expected shortfall.

Consider a probability triple $(\mathbf{\Omega},\mathcal{F}, \mathbb{P})$ and a time horizon $\Delta$. We then denote the set of all random variables which are almost surely finite on $(\mathbf{\Omega},\mathcal{F})$ as $L^0(\mathbf{\Omega},\mathcal{F}, \mathbb{P})$. A financial risk $\mathcal{M}$ is then said to be a subset of $L^0$, or $\mathcal{M} \subset L^0(\mathbf{\Omega},\mathcal{F}, \mathbb{P})$. 

$\mathcal{M}$ is also said to be a convex cone, meaning that given two risks (RVs) $X, Y \in \mathcal{M}$ and a scalar $a > 0$

$$X + Y \in \mathcal{M}$$

and 

$$aX \in \mathcal{M}$$


A risk measure $\mathcal{V}: \mathcal{M} \rightarrow \mathbb{R}$ is a function that maps a random variable $L \in \mathcal{M}$ to a real value. $\mathcal{V}(L)$ is interpreted as the amount of capital that must be added to the risk $L$ to make it acceptible for a decision maker. If $\mathcal{V}(L) > 0$ the risk is acceptable and $-\mathcal{V}(L)$ can be taken from the position without the risk becoming unacceptable. For example, if a bank finds that $\mathcal{V}(L) = -5000$EUR for one of its accounts, then one can withdraw up to 5000EUR without it causing any risk. If the measure was a positive number, say +5000EUR, then 5000 must be added into the account. 


The following is the set of axioms that a risk measure $\mathcal{V}: \mathcal{M} \rightarrow \mathbb{R}$ must satisfy to be a coherent risk measure:


**Axiom 1: Translational Invariance**

A risk measure $\mathcal{V}(L)$ is translationally invariant if for every risk $L \in \mathcal{M}$ and any amount of capital $l \in \mathbb{R}$

$$\mathcal{V}(L + l) = \mathcal{V}(L) + l$$

This means that by adding a safe amount of cash $l$, the amount of capital necessary to make the risk acceptable is reduced exactly by $l$.


**Axiom 2: Subadditivity**

For $L_1, L_2 \in \mathcal{M}$


$$\mathcal{V}(L_1 + L_2) \leq \mathcal{V}(L_1) + \mathcal{V}(L_2)$$

In other words, the composite risk of $L_1 + L_2$ will always be less than the sum of risks of $L_1$ and $L_2$. This statement axiomatically justifies diversification. Neglecting this axiom in portfolio optimization may lead to optimal portfolios being highly concentrated (and therefore risky).


**Axiom 3: Positive Homogeniety**

For all $L \in \mathcal{M}$ and every $\lambda > 0$


$$\mathcal{V}(\lambda L) = \lambda\mathcal{V}(L)$$

By adding a larger amount of capital at risk will correspond to a proportially scaled risk ratio (a simple example being CFDs). This may not be obvious because at a certain point of adding capital, there will be liquidity concerns.

**Axiom 4: Monotonicity**

For $L_1, L_2 \in \mathcal{M}$ such that $L_1 \leq L_2$,

$$\mathcal{V}(L_1) \leq \mathcal{V}(L_2)$$

This axiom becomes obvious if you state it as: positions that lead to higher losses require more risk capital for hedging.

**Definition 5: Coherent Risk Measure** 

Any risk measure $\mathcal{V}$ whose domain includes the convex cone $\mathcal{M}$ is said to be coherent on $\mathcal{M}$ iff Axioms 1-4 are satistfied.


**Note 6: Convex Risk Measures**

In axiom 3, I state that in practice, for large enough positions (i.e $\lambda >> 0$), there will be liquidity concerns. For example a large hedge fund trading penny stocks will corner the market and make the price react to the size of the trade itself. This implies $\mathcal{V}(\lambda L) > \lambda \mathcal{V}(L)$. It is for this reason that subadditivity and homeoginity definitions are treated loosely and sometimes requiring only a weaker property called convexity, or

$$\mathcal{V}(\lambda L_1 + (1 - \lambda)L_2) \leq \lambda\mathcal{V}(L_1) + (1 - \lambda)\mathcal{V}( L_2)$$

where $L_1, L_2 \in \mathcal{M}$ and $\lambda \in [0,1]$.

These axioms and concepts may seem highly abstract and theoretical but when first studied, the authors looked at various practiced risk measures and found flaws in many that did not satisfy the axioms above.


## Important Risk Measures



In [42]:
import cmath
def si(b,c):
    return np.array([[cmath.sqrt(1 - b*c), b],[c,-cmath.sqrt(1 - b*c)]])

In [76]:
si(-100,-100)

array([[   0.+99.99499987j, -100. +0.j        ],
       [-100. +0.j        ,   -0.-99.99499987j]])

In [77]:
si(0,0)

array([[ 1.+0.j,  0.+0.j],
       [ 0.+0.j, -1.-0.j]])

In [83]:
import sympy 
from sympy import Matrix, Symbol, sqrt

In [86]:
b = Symbol('b')
c = Symbol('c')
x = Symbol('x')
y = Symbol('y')
a = Matrix([[sqrt(1 - b*c), b],[c, - sqrt(1-b*c)]])
b = Matrix([[sqrt(1 - x*y), x],[y, - sqrt(1-x*y)]])


In [89]:
c = a@b

In [102]:
 c

Matrix([
[b*y + sqrt(-b*c + 1)*sqrt(-x*y + 1), -b*sqrt(-x*y + 1) + x*sqrt(-b*c + 1)],
[c*sqrt(-x*y + 1) - y*sqrt(-b*c + 1),  c*x + sqrt(-b*c + 1)*sqrt(-x*y + 1)]])

In [107]:
sqrt(1 - c[1]*c[2])

sqrt(-(-b*sqrt(-x*y + 1) + x*sqrt(-b*c + 1))*(c*sqrt(-x*y + 1) - y*sqrt(-b*c + 1)) + 1)